# mas

In [1]:
# load the questions 
from pathlib import Path
import json
from base64 import b64decode, b64encode

dataset = []

path = Path('data/set1')
questions = []
for file in path.glob('*.json'):
    with open(file, 'r') as f:
        data = json.load(f)
        # questions.extend(data['questions'])
        question = data.get('topic') 
        images = data.get('images') if data.get('images') else data.get('image')
        answer = data.get('answer')
        print(images)
        dataset.append({
            'id': file.as_posix(),
            'question': question,
            'images': [str(path/image) for image in images],
            'answer': answer
        })
        

['viscoelastic_parallel_1.png']
['static_equilibrium_1.png']
['long_jump_equivalent_approach.png']
['standing_man_problem.png']
['curvilinear_motion_1.png']
['standard_linear_solid_1.png']
['viscoelastic_exam_1.png']
['standing_man2_1.png', 'standing_man2_2.png', 'standing_man2_3.png']
['mass_moment_inertia_1.png']


In [2]:
from langchain_core.messages import HumanMessage
def build_message(data):
    question = data['question']
    images = data['images']
    answer = data['answer']
    
    
    content = []
    content.append({
        'type': 'text',
        'text': question
    })
    
    
    for image in images:
        with open(image, 'rb') as f:
            image_data = b64encode(f.read()).decode('utf-8')
            content.append({
                'type': 'image',
                'source_type': 'base64',
                'data': image_data,
                'mime_type': 'image/jpeg'
            })
    
    msg = HumanMessage(content=content)
    return msg

ModuleNotFoundError: No module named 'langchain_core'

In [ ]:
# build the mas
from src.solver import Solver,Manager, Reviewer,save_msg
from src.state import State
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model_name="gpt-4.1", temperature=0.6, max_tokens=4096)
llm = ChatOpenAI(model_name="qwem2.5-vl-72b-instruct",temperature=0.6,top_p = 0.95,base_url = 'http://127.0.0.1/1234')
solver = Solver(llm=llm)
manager = Manager(llm=llm)
reviewer = Reviewer(llm=llm)

from langgraph.graph import StateGraph,END

graph = StateGraph(state_schema=State)

# add nodes
graph.add_node(manager)
graph.add_node(solver)
graph.add_node(reviewer)

# add edges
def router(state: State):
    return state['send_to']

graph.add_conditional_edges(manager.__name__,router,path_map={
    'NEXT': solver.__name__,
    "END": reviewer.__name__,
})
graph.set_entry_point(manager.__name__)

graph.add_edge(solver.__name__, manager.__name__)
graph.add_edge(reviewer.__name__, END)

app = graph.compile()


In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
import os
os.environ["LANGGRAPH_MERMAID_DRAW_METHOD"] = "PYPPETEER"
display(Image(app.get_graph().draw_mermaid_png()))
# print(app.get_graph().draw_mermaid())   # should look like 'graph TD;\n START-->…'

In [ ]:
config = {"configurable":{'thread_id':'s'}}

from langchain_core.prompts import ChatPromptTemplate

for datas in dataset[2:]:

    human_message = build_message(datas)
    answer = dataset[0]['answer']
    human_message.pretty_print()
    for s in app.stream(input={
        'messages': [human_message],
        'ground_truth': answer,
    },config=config):
        try:
            for k,v in s.items():
                print(f'key: {k}')
                msgs = v
                
                for m in msgs['messages']:
                    # print(m.name)
                    # if m.name == k:
                    m.pretty_print()
                
                print(f'send_to: {msgs["send_to"]}')  
        except Exception as e:
            print(s)


    # save the messages
    md_file = datas['id'].replace('.json', '.md')
    with open(md_file, 'w') as f:
        for msg in msgs['messages']:
            if msg.__class__.__name__ == 'HumanMessage':
                continue
            f.write(f'# {msg.name}\n\n')
            f.write(msg.content)
            f.write('\n\n')
    f.close()

In [ ]:
msgs['messages'][0].__class__.__name__